In [104]:
from transformers import pipeline
import os
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
from tensorflow import keras
import os
from enum import Enum
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.patches as patches
import random
from tqdm import tqdm
import multiprocessing
import time

print("cwd", os.getcwd())

classifier = pipeline(model="facebook/detr-resnet-50")

cwd /home/linus/WashUCoding/DIS Copenhagen/Neural Networks/TomJerryProject/src


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Could not find image processor class in the image processor config or the model conf

In [106]:
base_path = "../archive/tom_and_jerry/tom_and_jerry"
folders = ["jerry", "tom", "tom_jerry_1"]

def process_images(image_list, result_list, folder): 
    # go over each image
    for i in tqdm(range(10)): 
    # for i in tqdm(range(len(image_list))): 
        print("starting image")
        currFrame = image_list[i]
        path_to_image = f"{base_path}/{folder}/{currFrame}"

        im = Image.open(path_to_image)
        res = classifier(im)

        # if boxes were predicted, created cropped images from each box
        if res: 
            boxes = []
            for j in range(len(res)): 
                boxCoords = res[j]["box"]["xmin"], res[j]["box"]["ymin"], res[j]["box"]["xmax"], res[j]["box"]["ymax"]
                boxes.append(boxCoords)
            # draw the box
            for j in range(len(boxes)): 
                xmin, ymin, xmax, ymax = boxes[j]
                width, height = boxCoords[2]-boxCoords[0], boxCoords[3]-boxCoords[1]
                croppedImage = im.crop((xmin, ymin, xmax, ymax))
                result_list.append((folder, croppedImage))
        print("finished with image")

In [109]:
# generate all cropped images
croppedImages = []
# go through each folder
for folder in folders: 
    currentFolder = base_path + "/" + folder
    pictures = os.listdir(currentFolder)

    resultList = []
    
    # process_images(pictures, resultList, folder)
    # croppedImages.extend(resultList)

    num_threads = 8
    process_list = []
    results = [[] for _ in range(num_threads)]

    # Split the list into 16 equal parts
    split_images = np.array_split(pictures, num_threads)

    # Create and start threads
    for i in range(num_threads):
        process = multiprocessing.Process(target=process_images, args=(split_images[i], results[i], folder))
        process_list.append(process)
        process.start()
        # time.sleep(1)
        print("Thread", i, "started")

    # Wait for all threads to finish
    for i, process in enumerate(process_list):
        process.join()
        print("Thread", i, "joined")

    print("Joining complete")
    for result in results: 
        print("Extending result")
        croppedImages.extend(result)
    print("COMPLETE")

Thread 0 started


Thread 1 started
Thread 2 started
Thread 3 started
Thread 4 started
Thread 5 started
Thread 6 started
Thread 7 started


KeyboardInterrupt: 

In [79]:
print(len(croppedImages))
# for j in range(20): 
#     print(croppedImages[j].shape)

16


In [62]:
# code found from: https://note.nkmk.me/en/python-pillow-add-margin-expand-canvas/
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

In [80]:
squarePaddedImgs = []
for j in range(len(croppedImages)): 
    folder, image = croppedImages[j]
    resizedImg = expand2square(image, (0, 0, 0))
    squarePaddedImgs.append((folder, resizedImg))
    

# TODO: Retrieve original frame's name, and append an underscore, e.g. "frame1035_1.jpg"
for j, folderImage in enumerate(squarePaddedImgs): 
    folder, image = folderImage
    image = image.resize((256, 256))
    image.save(f"../ObjectDetectedDataset/{folder}{j}.jpg")

In [ ]:
# COPY IN CASE I BREAK IT WITH PARALLELIZATION
base_path = "../archive/tom_and_jerry/tom_and_jerry"
folders = ["jerry", "tom", "tom_jerry_1"]

def process_images(image_list, result_list, folder): 
    # go over each image
    # for i in range(10): 
    for i in tqdm(range(len(image_list))): 
        currFrame = image_list[i]
        path_to_image = f"{base_path}/{folder}/{currFrame}"

        im = Image.open(path_to_image)
        res = classifier(im)
        # print("currentImage:", currFrame)
        # if boxes were predicted, created cropped images from each box
        if res: 
            boxes = []
            for j in range(len(res)): 
                # print(res[j]["box"])
                boxCoords = res[j]["box"]["xmin"], res[j]["box"]["ymin"], res[j]["box"]["xmax"], res[j]["box"]["ymax"]
                boxes.append(boxCoords)
            # print("All coords:", boxes)

            # fig, axis = plt.subplots()
            # axis.imshow(im)

            # draw the box
            for j in range(len(boxes)): 
                xmin, ymin, xmax, ymax = boxes[j]
                width, height = boxCoords[2]-boxCoords[0], boxCoords[3]-boxCoords[1]
                croppedImage = im.crop((xmin, ymin, xmax, ymax))
                result_list.append((folder, croppedImage))
                
                # croppedImage.show()
                # rect = patches.Rectangle((xmin, ymin), width, height, linewidth=1, edgecolor="w", facecolor="none", label="test")

                # axis.add_patch(rect)